In [1]:
import pandas as pd
import numpy as np

# Load the data
file_path = 'dr_results.csv'
data = pd.read_csv(file_path)

# Function to calculate width deviation for each interval
def calculate_awd(lower_bounds, upper_bounds):
    lower_bounds = np.array(eval(lower_bounds))
    upper_bounds = np.array(eval(upper_bounds))
    widths = upper_bounds - lower_bounds
    mean_width = np.mean(widths)
    width_deviation = np.abs(widths - mean_width)
    awd = np.mean(width_deviation)
    return awd

# Apply the function to each row
data['AWD'] = data.apply(lambda row: calculate_awd(row['Lower Bounds'], row['Upper Bounds']), axis=1)

# Save the updated dataframe to a new CSV file
updated_file_path = 'dr_results_with_awd.csv'
data.to_csv(updated_file_path, index=False)

print(f'Updated file saved to {updated_file_path}')


Updated file saved to dr_results_with_awd.csv


In [6]:
# Load the results
import ast
all_results_df = pd.read_csv("dr_results_with_awd.csv")

# Convert 'Lower Bounds' and 'Upper Bounds' from strings to lists
all_results_df['Lower Bounds'] = all_results_df['Lower Bounds'].apply(ast.literal_eval)
all_results_df['Upper Bounds'] = all_results_df['Upper Bounds'].apply(ast.literal_eval)

# Aggregate results and calculate average metrics
grouped = all_results_df.groupby(['Confidence Level', 'Model'])
avg_results = []

for name, group in grouped:
    avg_picp = group['PICP'].mean()
    avg_pinaw = group['PINAW'].mean()
    avg_ace = group['ACE'].mean()
    avg_awe = group['AWE'].mean()
    avg_awd = group['AWD'].mean()
    std_picp = group['PICP'].std()
    std_pinaw = group['PINAW'].std()
    std_ace = group['ACE'].std()
    std_awe = group['AWE'].std()
    std_awd = group['AWD'].std()
    
    lower_bounds = group['Lower Bounds'].apply(np.array).values
    upper_bounds = group['Upper Bounds'].apply(np.array).values
    
    lower_bounds_avg = np.mean(np.vstack(lower_bounds), axis=0).tolist()
    upper_bounds_avg = np.mean(np.vstack(upper_bounds), axis=0).tolist()
    
    avg_results.append({
        'Confidence Level': name[0],
        'Model': name[1],
        'Avg PICP': avg_picp,
        'Std PICP': std_picp,
        'Avg PINAW': avg_pinaw,
        'Std PINAW': std_pinaw,
        'Avg ACE': avg_ace,
        'Std ACE': std_ace,
        'Avg AWE': avg_awe,
         'Std AWE': std_awe,
        'Avg AWD' : avg_awd,
        'Std AWD' : std_awd,
        'Lower Bounds': lower_bounds_avg,
        'Upper Bounds': upper_bounds_avg,
    })

# Save the averaged results to a CSV file
avg_results_df = pd.DataFrame(avg_results)
avg_results_df.to_csv('avg_dr_results_with_awd.csv', index=False)
print("Results are saved to avg_dr_results_with_awd.csv")

Results are saved to avg_dr_results_with_awd.csv
